# Pasta Mouse

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
4. [Load features](#Load-features)
5. [Map to mouse orthologs](#Map-to-mouse-orthologs)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
9. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)


Let's first import some packages:

In [ ]:
import os
import inspect
import shutil
import json
import subprocess

import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [ ]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.Pasta)

In [ ]:
model = pya.models.Pasta()

## Define clock metadata

In [ ]:
model.metadata["clock_name"] = 'pasta_mouse'
model.metadata["data_type"] = 'transcriptomics'
model.metadata["species"] = 'Mus musculus'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = 'Salignon, Jerome, et al. "Pasta, an age-shift transcriptomic clock, maps the chemical and genetic determinants of aging and rejuvenation." bioRxiv (2025): 2025-06.'
model.metadata["doi"] = "https://doi.org/10.1101/2025.06.04.657785"
model.metadata["research_only"] = None
model.metadata["notes"] = 'Rank-normalized Pasta clock using mouse one-to-one ortholog genes when available.'

## Download clock dependencies

#### Download coefficient file

In [ ]:
coeff_url = "https://raw.githubusercontent.com/bio-learn/biolearn/master/biolearn/data/Pasta.csv"
os.system(f"curl -L {coeff_url} -o Pasta.csv")

#### Download ortholog mapping

In [ ]:
ortholog_url = "https://raw.githubusercontent.com/jsalignon/pasta/main/data/v_human_mouse_one2one.rda"
os.system(f"curl -L {ortholog_url} -o v_human_mouse_one2one.rda")

## Load features

#### From CSV file

In [ ]:
coeffs = pd.read_csv('Pasta.csv')
coeffs['feature'] = coeffs['GeneID']
coeffs['coefficient'] = coeffs['CoefficientTraining']

model.features = coeffs['feature'].tolist()

## Map to mouse orthologs

In [ ]:
r_cmd = (
    "load('v_human_mouse_one2one.rda'); "
    "df <- data.frame(mouse=names(v_human_mouse_one2one), human=as.character(v_human_mouse_one2one)); "
    "write.csv(df, 'v_human_mouse_one2one.csv', row.names=FALSE)"
)
os.system(f"Rscript -e \"{r_cmd}\"")

ortholog_df = pd.read_csv('v_human_mouse_one2one.csv')
human_to_mouse = dict(zip(ortholog_df['human'], ortholog_df['mouse']))

mapped_features = [human_to_mouse.get(gene, gene) for gene in model.features]
mapped_count = sum(gene in human_to_mouse for gene in model.features)
print(f"Mapped {mapped_count} of {len(model.features)} features to mouse orthologs.")
model.features = mapped_features

## Load weights into base model

#### From CSV file

In [ ]:
weights = torch.tensor(coeffs['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([0.0])

#### Linear model

In [ ]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [ ]:
model.reference_values = [float('nan')] * len(model.features)

## Load preprocess and postprocess objects

In [ ]:
model.preprocess_name = "median_fill_and_rank_normalization"
model.preprocess_dependencies = None

In [ ]:
model.postprocess_name = "scale_and_shift"
model.postprocess_dependencies = [-4.76348378687217, -0.0502893445253186]

## Check all clock parameters

In [ ]:
pya.utils.print_model_details(model)

## Basic test

In [ ]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

## Save torch model

In [ ]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [ ]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)